In [ ]:
!pip3 install nnfs
!pip3 install icecream

In [ ]:
from nnfs.datasets import spiral_data
import numpy as np
import nnfs
from icecream import ic
np.random.seed(0)

In [ ]:

class Layer_Dense:
    def __init__(self, input_params: int, number_of_neurons: np.int32):
        self.weights=np.random.randn(input_params, number_of_neurons) * 0.01
        self.bias=np.zeros(number_of_neurons)

    def forward(self, input_matrix):
        self.output=np.dot(input_matrix, self.weights)+self.bias
        self.inputs=input_matrix
        return self.output

    def backward(self, dvalues):
        self.dweights=np.dot(self.inputs.T, dvalues)
        self.dbias=ic(np.sum(dvalues, axis=0, keepdims=False))

        self.dinputs=np.dot(dvalues, self.weights.T)


In [ ]:
arr=ic(np.random.rand(3,3)*100//10)
ic(arr-ic(np.max(arr, axis=0, keepdims=False)))

ic| np.random.rand(3,3)*100//10: array([[5., 7., 6.],
                                        [5., 4., 6.],
                                        [4., 8., 9.]])
ic| np.max(arr, axis=0, keepdims=False): array([5., 8., 9.])
ic| arr-ic(np.max(arr, axis=0, keepdims=False)): array([[ 0., -1., -3.],
                                                        [ 0., -4., -3.],
                                                        [-1.,  0.,  0.]])


array([[ 0., -1., -3.],
       [ 0., -4., -3.],
       [-1.,  0.,  0.]])

In [ ]:
np.max(arr, axis=1, keepdims=1)
np.max(arr, axis=0, keepdims=1)

array([[5., 8., 9.]])

In [ ]:
class SoftMax:
    def forward(self, inputs):
        rowMax=np.max(inputs, keepdims=1, axis=1)
        expVals=np.exp(ic(inputs)-rowMax)
        baseSums=np.sum(expVals, keepdims=1, axis=1)
        probabilites=expVals/baseSums
        self.output=probabilites
        return probabilites

class Relu:
    def forward(self, inputs):
        self.inputs=inputs
        self.output=np.maximum(0, inputs)
    def backward(self, dvalues):
        self.dinputs=dvalues.copy()
        self.dinputs[self.dinputs<=0]=0



In [ ]:
class Loss:
    def calculate(self, output, y):
        sample_losses=self.forward(output, y)
        data_loss=np.sum(sample_losses
                         )
        return data_loss

In [ ]:
# Cross Entropy loss
class Loss_Categorical_CrossEntropy(Loss):

    def forward(self, y_pred, y_true):
        y_pred_clipped=np.clip(y_pred, 1e-7, 1 - 1e-7)
        samples=len(y_pred)
        if len(y_true.shape)==1:
            correct_confidence=y_pred_clipped[range(samples), y_true]

        elif len(y_true.shape)==2:
            correct_confidence=np.sum(y_pred_clipped*y_true, axis=1)

        negative_log_likelyhood=-np.log(correct_confidence)
        return negative_log_likelyhood

In [ ]:
class Activation_Softmax_Loss_CategoricalCrossEntropy:
    def __init__(self):
        self.activation=SoftMax()
        self.loss=Loss_Categorical_CrossEntropy()
    def forward(self, inputs, y_true):
        self.inputs=inputs
        self.activation.forward(inputs)
        self.output=self.activation.output
        return self.loss.calculate(self.output, y_true)
    def backward(self, dvalues, y_true):
        samples=len(dvalues)
        if samples==2:
            y_true=np.argmax(y_true, axis=1)

        self.dinputs=dvalues.copy()
        self.dinputs[range(samples), y_true]-=1
        self.dinputs=self.inputs/samples


In [ ]:
import nnfs
from nnfs.datasets import spiral_data
X, y= spiral_data(samples=100, classes=3)


In [ ]:
class Optimizer_SDG:
    def __init__(self, learning_rate=0.01, decay=0.0):
        self.learning_rate=learning_rate
        self.current_learning_rate=learning_rate
        self.decay=decay
        self.iterations=0

    def update_params(self, layer:Layer_Dense):
        layer.weights += -self.learning_rate *layer.dweights
        layer.bias += -self.learning_rate * layer.dbias
    def pre_update_params(self):
        if self.decay:
            self.learning_rate * \
                             (1. / (1. + self.decay * self.iterations))

    def post_update_params(self):
        self.iterations+=1

# PlayGround

In [ ]:
dense1 = Layer_Dense(2, 64)
activation1 = Relu()

dense2 = Layer_Dense(64, 3)

loss_activation = Activation_Softmax_Loss_CategoricalCrossEntropy()

optimizer = Optimizer_SDG(.1, decay=1e-3)

# performing iterations
ic.disable()
for epoch in range(100001):
    dense1.forward(X)
    activation1.forward(dense1.output)
    dense2.forward(activation1.output)

    loss = ic(loss_activation.forward(dense2.output, y))

    predictions = np.argmax(loss_activation.output, axis=1)

    if len(y.shape) == 2:
        y=np.argmax(y, axis=1)

    accuracy = np.mean(predictions == y)
    if epoch % 1000 == 0:
        ic.enable()
        ic(f'epoch: {epoch} accuracy: {accuracy} loss: {loss}')
        ic.disable()


    loss_activation.backward(loss_activation.output, y)
    dense2.backward(loss_activation.dinputs)
    activation1.backward(dense2.dinputs)
    dense1.backward(activation1.dinputs)

    #update
    # ic(dense1.bias)
    # ic(dense1.dbias)
    optimizer.pre_update_params()
    optimizer.update_params(dense1)
    optimizer.update_params(dense2)
    optimizer.post_update_params()

ic| f'epoch: {epoch} accuracy: {accuracy} loss: {loss}': 'epoch: 0 accuracy: 0.2833333333333333 loss: 329.5942377780001'
ic| f'epoch: {epoch} accuracy: {accuracy} loss: {loss}': 'epoch: 1000 accuracy: 0.2866666666666667 loss: 329.59331506302516'
ic| f'epoch: {epoch} accuracy: {accuracy} loss: {loss}': 'epoch: 2000 accuracy: 0.2866666666666667 loss: 329.5924844207826'
ic| f'epoch: {epoch} accuracy: {accuracy} loss: {loss}': 'epoch: 3000 accuracy: 0.29333333333333333 loss: 329.59172316399327'
ic| f'epoch: {epoch} accuracy: {accuracy} loss: {loss}': 'epoch: 4000 accuracy: 0.29 loss: 329.5910591382851'
ic| f'epoch: {epoch} accuracy: {accuracy} loss: {loss}': 'epoch: 5000 accuracy: 0.29 loss: 329.5904648618805'
ic| f'epoch: {epoch} accuracy: {accuracy} loss: {loss}': 'epoch: 6000 accuracy: 0.2833333333333333 loss: 329.58994704676434'
ic| f'epoch: {epoch} accuracy: {accuracy} loss: {loss}': 'epoch: 7000 accuracy: 0.29 loss: 329.58947863985384'
ic| f'epoch: {epoch} accuracy: {accuracy} loss: 

In [ ]:
ic.disable()
dense1= Layer_Dense(2, 3)
activation1= Relu()
dense2=Layer_Dense(3, 3)
loss_activation= Activation_Softmax_Loss_CategoricalCrossEntropy()
dense1.forward(X)
activation1.forward(dense1.output)
dense2.forward(activation1.output)
loss=loss_activation.forward(dense2.output, y)
loss_activation.output[:5]

predictions=np.argmax(loss_activation.output, axis=1)
predictions
if len(y.shape) == 2:
    y=np.argmax(y, axis=1)
ic.enable()
ic(y)
accuracy=np.mean(predictions==y)
accuracy

ic| y: array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
              0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
              0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
              0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
              0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
              1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
              1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
              1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
              1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
              1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
              2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
              2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
              2, 2, 2, 2, 2, 2, 2, 2, 2,

np.float64(0.30666666666666664)

In [ ]:
sample_output=np.array([
    [.7, .1, .2],
    [.1, .5, .4],
    [.1, .9, .01]
])

class_targets = np.array([
    [1,0,0],
    [0,1,0],
    [0, 1, 0]
])
np.sum(sample_output, axis=1, keepdims=True)


In [ ]:
weights=np.array([[-3.0, -1.0, 2],[2.0, -2.0, 3.0], [1.0, 2.1, -2.0]])
bias=[2, 3]
inputs=np.array([1.0, -2.0, 3])
target_output=[0,1]
learning_rate=0.001

def relu(x):
    return np.maximum(0, x)

def relu_derivative(x):
    return np.where(x>0, 1.0 ,0.0 )
ic.enable()

def CalculateLoss(output, target_output):
    return (output-target_output)**2

for i in range(100):
    # forward pass
    linear_output = ic(np.dot(inputs, weights))+bias
    output=ic(relu(linear_output))
    loss = CalculateLoss(output, target_output)
    ic(loss)

    # Backward pass
    Dloss_Doutput=(output-target_output)*2
    Doutput_Dlinear=relu_derivative(linear_output)
    Dlinear_Dweights=inputs
    Dlinear_Dbias=1.0

    Dloss_Dlinear = ic(Dloss_Doutput * Doutput_Dlinear)
    Dloss_Dweights = ic(np.outer(Dloss_Dlinear, Dlinear_Dweights))
    Dloss_Dbias = Dloss_Dlinear * Dlinear_Dbias

#     # Update weights

    weights-= learning_rate*Dloss_Dweights
    bias -=learning_rate*Dloss_Dbias

    deltaLoss=(loss-preLoss)/preLoss
    print(f"iterations: {i+1} loss: {np.sum(loss)}, deltaLoss: {deltaLoss}")
    preLoss=loss

# weights, bias